In [3]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Get stock data
stock = yf.Ticker("AAPL")
hist = stock.history(period="5y")

# Calculate indicators
hist['Volume Change'] = hist['Volume'].pct_change().apply(lambda x: 1 if x > 0 else 0)
hist['Close Change'] = hist['Close'].pct_change()
hist['Open Change'] = hist['Open'].pct_change()
hist['High Change'] = (hist['High'] - hist['Open']) / hist['Open']
hist['Low Change'] = (hist['Low'] - hist['Open']) / hist['Open']
hist['CO Change'] = (hist['Close'] - hist['Open']) / hist['Open']

# Categorize changes
def categorize_change(change):
    if change < -0.02:
        return 0
    elif -0.02 <= change < -0.005:
        return 1
    elif -0.005 <= change <= 0.005:
        return 2
    elif 0.005 < change <= 0.02:
        return 3
    else:
        return 4

def categorize_high_change(change):
    if change < 0.005:
        return 0
    elif 0.005 <= change <= 0.02:
        return 1
    else:
        return 2

def categorize_low_change(change):
    if -0.005 <= change <= 0:
        return 0
    elif -0.02 <= change < -0.005:
        return 1
    else:
        return 2

hist = hist.dropna()
hist['Close Change'] = hist['Close Change'].apply(categorize_change)
hist['Open Change'] = hist['Open Change'].apply(categorize_change)
hist['High Change'] = hist['High Change'].apply(categorize_high_change)
hist['Low Change'] = hist['Low Change'].apply(categorize_low_change)
hist['CO Change'] = hist['CO Change'].apply(categorize_change)

# Prepare dataset
features = ['Volume Change', 'Close Change', 'Open Change', 'High Change', 'Low Change', 'CO Change']
X = hist[features].iloc[:-1]
y = hist[features].shift(-1).dropna()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X[:-1], y[:-1], test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Print results for each label
for i, feature in enumerate(features):
    print(f"Classification report for {feature}:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))
    print("-" * 80)

Classification report for Volume Change:
              precision    recall  f1-score   support

         0.0       0.64      0.55      0.59       142
         1.0       0.51      0.61      0.55       109

    accuracy                           0.57       251
   macro avg       0.58      0.58      0.57       251
weighted avg       0.59      0.57      0.58       251

--------------------------------------------------------------------------------
Classification report for Close Change:
              precision    recall  f1-score   support

         0.0       0.20      0.19      0.19        32
         1.0       0.20      0.19      0.20        53
         2.0       0.28      0.34      0.31        65
         3.0       0.25      0.28      0.27        67
         4.0       0.06      0.03      0.04        34

    accuracy                           0.23       251
   macro avg       0.20      0.21      0.20       251
weighted avg       0.22      0.23      0.22       251

----------------------